### Imports

In [1]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential, load_model
from keras.layers import Dense, Bidirectional
from keras.layers.recurrent import GRU

from encrypt import *
from tools import *
import string

Using TensorFlow backend.


### Configurations

In [2]:
letters = string.printable.split('!')[0]
encrypt = transposition_cipher('25741863')
samples_count = 200000
text_length = 8

set_characters(letters)
model_path = f'models/{encrypt.name}_{len(letters)}x{text_length}_best_model.h5'

### Generating and Preparing Data

In [3]:
text = generate_text(text_length, samples_count)
cipher = list(map(encrypt, text))

train_text = to_vec(text)
train_cipher = to_vec(cipher)

### Building and Training the Model

In [4]:
model = Sequential()
model.add(Bidirectional(GRU(128, activation='relu', return_sequences=True), input_shape=train_cipher.shape[1:]))
model.add(Dense(1, activation='linear'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 8, 256)            99840     
_________________________________________________________________
dense_1 (Dense)              (None, 8, 1)              257       
Total params: 100,097
Trainable params: 100,097
Non-trainable params: 0
_________________________________________________________________


In [5]:
callbacks = [
    EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=3, verbose=1, mode='min'),
    ModelCheckpoint(model_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
]
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.fit(train_cipher, train_text, validation_split=.1, batch_size=10, epochs=5, callbacks=callbacks)

Train on 180000 samples, validate on 20000 samples
Epoch 1/5
180000/180000 [==============================] - 148s 821us/step - loss: 6.4031 - acc: 0.7031 - val_loss: 0.0453 - val_acc: 0.9760

Epoch 00001: val_acc improved from -inf to 0.97597, saving model to models/transposition_cipher_62x8_best_model.h5
Epoch 2/5
180000/180000 [==============================] - 147s 818us/step - loss: 0.0513 - acc: 0.9619 - val_loss: 0.0091 - val_acc: 0.9997

Epoch 00002: val_acc improved from 0.97597 to 0.99968, saving model to models/transposition_cipher_62x8_best_model.h5
Epoch 3/5
180000/180000 [==============================] - 145s 808us/step - loss: 0.0301 - acc: 0.9825 - val_loss: 0.0035 - val_acc: 0.9999

Epoch 00003: val_acc improved from 0.99968 to 0.99994, saving model to models/transposition_cipher_62x8_best_model.h5
Epoch 4/5
180000/180000 [==============================] - 146s 811us/step - loss: 0.0230 - acc: 0.9895 - val_loss: 0.0031 - val_acc: 0.9999

Epoch 00004: val_acc did not i

### Predicting and Evaluating the Model

In [6]:
model = load_model(model_path)

In [7]:
test_cyhper = encrypt('HeyHello')
prediction = to_txt(model.predict(to_vec(test_cyhper)))

print('Model predicted that original text is:', prediction)
print('Test cipher vs encrypted prediction:', test_cyhper, 'vs', encrypt(prediction))

Model predicted that original text is: HeyHello
Test cipher vs encrypted prediction: lHeeoHyl vs lHeeoHyl
